<a href="https://colab.research.google.com/github/pepborrell/LauzHack2019/blob/master/LauzHack2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coses random de bash

In [0]:
# Install client library
!pip3 install --upgrade google-cloud-vision

In [0]:
!export GOOGLE_APPLICATION_CREDENTIALS="./LauzHack2019-81776abcfeb4.json"

In [0]:
!rm -r meme_json
!rm -r memes
!mkdir meme_json
!mkdir memes

# Lectura de url's

In [0]:
# read urls from csv

import pandas as pd

#url = 'https://pastebin.com/raw/SjMqw5Wx' # 10.000 memes
#url = 'https://pastebin.com/raw/x5PMM3Dh' # 100 memes amb molts upvotes
#url = 'https://pastebin.com/raw/nxSK0mJV' #1000 mems entre 50 i 1000 upvotes
url = 'https://pastebin.com/raw/MHtpZScX' # 2000 mems entre 400 i 800 upvotes

df = pd.read_csv(url)



In [38]:
# download memes with url list from df to local folder

import requests
import threading
import io

N_THREADS = 20
MAX_MEMES = 2000

def get_memes(segment):
  start = int(segment*(MAX_MEMES/N_THREADS))
  end = min(start + int(MAX_MEMES/N_THREADS), df.shape[0])
  count = start

  for row in df.itertuples():
    #print(row)
    if row.Index < start:
      continue
    elif row.Index >= end:
      break
    
    upvotes = row.upvotes
    imageUrl = row.url
    #print("upvotes: {}, url: {}".format(upvotes, imageUrl))
    fname = imageUrl.split("/")[-1]
    fname2 = str(upvotes) + "___" + fname
    try:
      res = requests.get(imageUrl)
      io.open("./memes/" +fname2, "wb").write(res.content)
    except:
      print("Err")
    print(count)
    count +=1

threads = []
for tn in range(N_THREADS):
  x = threading.Thread(target=get_memes, args=(tn,))
  x.start()
  threads.append(x)
for thread in threads:
  thread.join()

print("Finished")


0
400
700
800
1000
1
300
500
1500
1100
2
13001600
1200
100

1700
1800
1400
600900

701
3
401
1900
801
200
501
4
301
1101
5
1201
1801
16011701

13011501

1401
901601

6
702
402
201
1901
302
802
1001
1102
1202
502
1802
1702
1402
7
6021302

202
1602
703
303
1902
902
403
1502
1002
1803
1203
1703
8
1103
603
1403
1303
704
503
903
101
1903
1503
404
803
9
1804
1204
1404
1304
604
304
1104
1704
203
504705
10

1003
102
1504
1603
1904
405
1805
1205
804
605
11
1705
1105
1305
1405
1905
1604
305706

12
1004
1505
505
1206
103
204
1806
131706

8051106

606
406
904
1306
1406
1005
1906
506
1506
1207
14
306
104
806
507
70717071807


151605
905

1407
1107
1907
607
1006205

1208
307
105
1507
16
407
807
708
508
17
1307
1808
1708
906
1108608

18
308
1408
1908
408
808
106
1007206
1508

709
1209
1709
907
1109
1308
509
609
1409
1809
1909
1008
1210
1509
809
409
710
309
1710
1606
908
1110
510
207
1410
1810
610
1510
19101309

810
711
1009
310
1211
1607
909
1711
1111
208
611
511
410
1411
1511
712
811
18111310

311
9

##Neteja de JSONS:

In [0]:
import json
from collections import namedtuple 
import pandas as pd
MemeData = namedtuple('MemeData', ["upvotes","annotations", "text",   "matchparcial",
                                    "matchpages", "matchsimilar", "webLabels"])
def processGoogleResult(meme, upvotes):
  # Annotations
  Annotation = namedtuple('Annotation', ['name', 'score'])
  annotations = []
  for ann in meme.get("labelAnnotations",[]):
    annotations.append(Annotation(ann["description"],ann["score"]))

  # meme text
  text = meme.get("fullTextAnnotation",{}).get("text","")

  # Web entities
  webthings = meme.get("webDetection",{})
  for ent in webthings.get("webEntities",[]):
    try:
      annotations.append(Annotation(ent["description"],ent["score"]))
    except:
      pass
  matchparcial = len(webthings.get("partialMatchingImages",[]))
  matchpages = len(webthings.get("pagesWithMatchingImages",[]))
  matchsimilar = len(webthings.get("visuallySimilarImages",[]))
  webLabels = []
  for label in webthings["bestGuessLabels"]:
    try:
      webLabels.append(label["label"])
    except:
      pass
  return MemeData(upvotes, annotations, text,  matchparcial, matchpages,
                matchsimilar, webLabels)

##Crides a api de google

In [40]:
from google.protobuf.json_format import MessageToJson
from google.cloud import vision
from google.cloud.vision import enums
from google.cloud.vision import types
import threading
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="./LauzHack2019-81776abcfeb4.json"
def detect_data(in_path):
    """Detects labels in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    features = [
      types.Feature(type=enums.Feature.Type.TEXT_DETECTION),
      types.Feature(type=enums.Feature.Type.LABEL_DETECTION),
      types.Feature(type=enums.Feature.Type.WEB_DETECTION),
    ]
    if os.stat(in_path).st_size > 1024*1024*4:
      return -1
    with io.open(in_path, 'rb') as image_file:

        content = image_file.read()

    image = vision.types.Image(content=content)

    req = types.AnnotateImageRequest(image=image, features=features)
    response = client.annotate_image(req)
    upvotes = int(in_path.split("/")[-1].split("___")[0])
    #response = client.label_detection(image=image)
    return processGoogleResult(json.loads(MessageToJson(response)), upvotes)


from os import listdir
from os.path import isfile, join

in_path = "./memes/"
out_path = "./meme_json/"

onlyfiles = [f for f in listdir(in_path) if isfile(join(in_path, f))]


N_THREADS = 20
MAX_MEMES = 2000
memes_tmp = [[]for x in range(N_THREADS)]

def work_bitch(segment):
  start = int(segment* (MAX_MEMES/N_THREADS))
  end = start + int(MAX_MEMES/N_THREADS)
  count = start
  for f in onlyfiles[start:end]:
    print("Generating json: {}".format(count))
    resdata = detect_data(join(in_path, f))
    if resdata == -1:
      print("Too big")
      continue
    memes_tmp[segment].append(resdata)
    count += 1

threads = []
for tn in range(N_THREADS):
  x = threading.Thread(target=work_bitch,args=(tn,))
  x.start()
  threads.append(x)
for thread in threads:
  thread.join()

memes = []
for tn in range(N_THREADS):
  memes += memes_tmp[tn]
print("Finished")

Generating json: 0
Generating json: 100
Generating json: 200
Generating json: 300Generating json: 400

Generating json: 600Generating json: 700Generating json: 800

Generating json: 500

Generating json: 900
Generating json: 1000
Generating json: 1100
Generating json: 1200
Generating json: 1300Generating json: 1400

Generating json: 1500
Generating json: 1600Generating json: 1700Generating json: 1800


Generating json: 1900
Generating json: 701
Generating json: 301
Generating json: 1801
Generating json: 1701
Generating json: 401
Generating json: 501
Generating json: 601Generating json: 1301

Generating json: 801
Generating json: 1
Generating json: 201
Generating json: 1501Generating json: 1401Generating json: 1601Generating json: 1001


Generating json: 1201

Generating json: 101
Generating json: 901
Generating json: 1901Generating json: 1101

Generating json: 702
Generating json: 502
Generating json: 2
Generating json: 402Generating json: 1802

Generating json: 302
Generating json: 17

In [41]:
memes2 = []
for meme in memes:
  newanon = [[a.name,a.score] for a in meme.annotations]
  memes2.append(MemeData(meme.upvotes, newanon, meme.text,  meme.matchparcial, meme.matchpages,
                meme.matchsimilar, meme.webLabels))
df = pd.DataFrame.from_records(
   memes2,
   columns=MemeData._fields
)
print("Generating...")
df.to_csv("memes4.csv")
print(df.shape)
df.head()

Generating...
(1996, 7)


,upvotes,annotations,text,matchparcial,matchpages,matchsimilar,webLabels
0,688,"[[Water, 0.9100271463394165], [Text, 0.8692919...",When a girl\nsees my\nshrek dvd collection\n,10,10,10,[girl water funny]
1,453,"[[Photo caption, 0.6833236813545227], [Photogr...",Man with Alzheimer's forgets he had\nAlzheimer...,0,1,10,[cartoon]
2,488,"[[Vehicle, 0.9078187346458435], [Car, 0.877487...",Who...slow down there redditor. Do you know\nh...,10,10,10,[nz police speeding]
3,535,"[[Adaptation, 0.7408301830291748], [Tree, 0.68...",Michelle: wait why are we pulling over?\nBarac...,10,10,10,[obama 69]
4,601,"[[Cat, 0.9928472638130188], [Mammal, 0.9727270...",ME\nLast mission of my\nfavourite game\n,10,10,2,[me last mission of game]


# Random script to waste time executing

In [0]:
import time
ii = 0

while True:
  print(ii)
  time.sleep(5)
  ii += 1

    